In [5]:
import sys
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install opencv-python
# !{sys.executable} -m pip install tensorflow
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install sklearn
import os
import cv2
import pandas as pd
import tensorflow as tf
import warnings
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras import layers
from tensorflow.keras.models import Model

In [3]:

train_folder = './Dataset/train'
test_folder = './Dataset/test'
valid_folder = './Dataset/valid'

# Function to preprocess the dataset
def preprocess_dataset(folder_path):
    warnings.filterwarnings("ignore", category=UserWarning, module="cv2")
    images = []
    label = []
    csv_path = os.path.join(folder_path, '_classes.csv')
    df = pd.read_csv(csv_path)

    df.columns = df.columns.str.replace(' ', '')
    for _, row in df.iterrows():
        try:
            filename = row['filename']
            image_path = os.path.join(folder_path, filename)
            image = cv2.imread(image_path)

            image = cv2.resize(image, (224, 224))
            image = image / 255.0
            
            label_index = (row.iloc[1:] == 1).idxmax() 
            numeric_index = list(row.index).index(label_index)
            if(label_index == 'Unlabeled'):
                continue
            images.append(image)
            label.append(numeric_index)
        except: 
            continue
    converted_data = tf.data.Dataset.from_tensor_slices((images, label)).batch(16)
    return converted_data


In [6]:

train_dataset_tf = preprocess_dataset(train_folder)
test_dataset_tf = preprocess_dataset(test_folder)
valid_dataset_tf = preprocess_dataset(valid_folder)

[ WARN:0@7.272] global loadsave.cpp:248 findDecoder imread_('./Dataset/test/canvas_1629911700_jpg.rf.54685d79d3d987dec59c9d82a03ea21a.jpg'): can't open/read file: check file path/integrity
[ WARN:0@7.274] global loadsave.cpp:248 findDecoder imread_('./Dataset/test/canvas_1629903174_jpg.rf.569f4025a4d51a1f780b2f7dae31c051.jpg'): can't open/read file: check file path/integrity
[ WARN:0@7.280] global loadsave.cpp:248 findDecoder imread_('./Dataset/test/canvas_1629911897_jpg.rf.5873c86fa1d18181d0269e3447bbcc88.jpg'): can't open/read file: check file path/integrity
[ WARN:0@7.286] global loadsave.cpp:248 findDecoder imread_('./Dataset/test/canvas_1629924022_jpg.rf.5b18df277f0f4a227c5d7de2cb5b87b7.jpg'): can't open/read file: check file path/integrity
[ WARN:0@7.287] global loadsave.cpp:248 findDecoder imread_('./Dataset/test/canvas_1629999768_jpg.rf.5c66dec94db9153bcf386ee0ed29248a.jpg'): can't open/read file: check file path/integrity
[ WARN:0@7.292] global loadsave.cpp:248 findDecoder imr

In [37]:


def AlexNet():
  inp = layers.Input((224, 224, 3)) #  Input layer of size (224x224) with 3 color channels
  x = layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu')(inp) # 
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling2D(3, 2)(x)
  x = layers.Conv2D(256, 5, 1, activation='relu')(x)
  x = layers.BatchNormalization()(x)
  x = layers.MaxPooling2D(3, 2)(x)
  x = layers.Conv2D(384, 3, 1, activation='relu')(x)
  x = layers.Conv2D(384, 3, 1, activation='relu')(x)
  x = layers.Conv2D(256, 3, 1, activation='relu')(x)
  x = layers.MaxPooling2D(3, 2)(x)
  x = layers.Flatten()(x)
  x = layers.Dense(4096, activation='relu')(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(4096, activation='relu')(x)
  x = layers.Dropout(0.5)(x)
  x = layers.Dense(7, activation='softmax')(x)

  model = Model(inputs=inp, outputs=x)

  return model

model = AlexNet()
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d_15 (Conv2D)          (None, 54, 54, 96)        34944     
                                                                 
 batch_normalization_6 (Bat  (None, 54, 54, 96)        384       
 chNormalization)                                                
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 26, 26, 96)        0         
 g2D)                                                            
                                                                 
 conv2d_16 (Conv2D)          (None, 22, 22, 256)       614656    
                                                                 
 batch_normalization_7 (Bat  (None, 22, 22, 256)       1024

In [38]:

import tensorflow.keras as keras

optimizer = keras.optimizers.legacy.Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer,
              loss="sparse_categorical_crossentropy", metrics=['accuracy'])

In [32]:
from tensorflow.keras.callbacks import EarlyStopping
# es = EarlyStopping(patience=5, 
#                    monitor='loss')
es = ReduceLROnPlateau(monitor='val_loss', patience = 5, verbose=1,factor=0.5, min_lr=0.00001)


In [39]:
model.fit(train_dataset_tf, epochs=10, validation_data=valid_dataset_tf,
          callbacks=[es], batch_size=32)

Epoch 1/10


384/384 [==============================] - 94s 244ms/step - loss: 1.7735 - accuracy: 0.2270 - val_loss: 1.7540 - val_accuracy: 0.2698 - lr: 1.0000e-04
Epoch 2/10
384/384 [==============================] - 109s 283ms/step - loss: 1.6788 - accuracy: 0.2950 - val_loss: 1.6285 - val_accuracy: 0.3474 - lr: 1.0000e-04
Epoch 3/10
384/384 [==============================] - 105s 272ms/step - loss: 1.5871 - accuracy: 0.3464 - val_loss: 1.6759 - val_accuracy: 0.3210 - lr: 1.0000e-04
Epoch 4/10
384/384 [==============================] - 96s 251ms/step - loss: 1.4749 - accuracy: 0.4090 - val_loss: 1.6543 - val_accuracy: 0.2990 - lr: 1.0000e-04
Epoch 5/10
384/384 [==============================] - 104s 271ms/step - loss: 1.3749 - accuracy: 0.4559 - val_loss: 1.6305 - val_accuracy: 0.3373 - lr: 1.0000e-04
Epoch 6/10
384/384 [==============================] - 98s 255ms/step - loss: 1.2525 - accuracy: 0.5147 - val_loss: 1.5995 - val_accuracy: 0.3581 - lr: 1.0000e-04
Epoch 7/10
384/384 [================

In [40]:
model.save('location_checker.h5')

/Users/eldrian/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [41]:
# Testing
model.fit(train_dataset_tf, epochs=5, validation_data=test_dataset_tf,
          callbacks=[es], batch_size=32)


Epoch 1/5
384/384 [==============================] - 107s 277ms/step - loss: 0.5449 - accuracy: 0.8026 - val_loss: 2.2218 - val_accuracy: 0.3534 - lr: 1.0000e-04
Epoch 2/5
384/384 [==============================] - 103s 267ms/step - loss: 0.4252 - accuracy: 0.8503 - val_loss: 3.5446 - val_accuracy: 0.3129 - lr: 1.0000e-04
Epoch 3/5
384/384 [==============================] - 105s 274ms/step - loss: 0.3559 - accuracy: 0.8738 - val_loss: 2.8234 - val_accuracy: 0.3558 - lr: 1.0000e-04
Epoch 4/5
384/384 [==============================] - 107s 278ms/step - loss: 0.2966 - accuracy: 0.8919 - val_loss: 2.4865 - val_accuracy: 0.4074 - lr: 1.0000e-04
Epoch 5/5
384/384 [==============================] - 95s 247ms/step - loss: 0.2344 - accuracy: 0.9199 - val_loss: 3.6344 - val_accuracy: 0.3166 - lr: 1.0000e-04


In [8]:
from tensorflow.keras.models import load_model

model = load_model('location_checker.h5')

test_loss, test_accuracy = model.evaluate(test_dataset_tf)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

51/51 [==============================] - 3s 60ms/step - loss: 2.0375 - accuracy: 0.3939
Test Loss: 2.037536382675171
Test Accuracy: 0.39386501908302307
